In [31]:
import requests
import time
from dotenv import load_dotenv
import pandas as pd

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup

## 

---

In [23]:
# Initailize
jobDescription = ["software",
                  "software intern",
                  "software development intern",
                  "software developer",
                  "SDET",
                  "軟體實習生",
                  "軟體測試實習生",
                  "data analytics",
                  "data science"]


In [30]:
def containOrNot(job, jobDescription: list):
    """
    
    """
    tokenList = job.get_attribute("data-job-name").split(" ")
    for token in tokenList:
        # if token in jobDescription:
        #     return True
        # else:
        #     return False
        flag = True if token in jobDescription else False
        return flag

In [33]:
driver = webdriver.Chrome()

# 進入104 首頁
# driver.get("https://www.google.com/")
driver.get("https://www.104.com.tw/jobs/main/")
driver.implicitly_wait(2) # Loading wait⋯

# 輸入要查詢的工作內容
keyWords = driver.find_element(By.CLASS_NAME, "form-control")
keyWords.send_keys("software intern")
searchBtn = driver.find_element(By.CLASS_NAME, "btn-secondary")
searchBtn.click()

# 開始搜索符合條件的工作
jobs = [] # array to store dict results
nextPageBtn = driver.find_element(By.CLASS_NAME, "js-next-page")
while nextPageBtn:
    jobs_parent = driver.find_element(By.ID, "js-job-content")
    jobs_per_page = jobs_parent.find_elements(By.TAG_NAME, "article")
    for job in jobs_per_page:
        if containOrNot(job, jobDescription):
            jobs.append({
                "name": job.get_attribute("data-job-name"),
                "company": job.get_attribute("data-cust-name")
            }) 
            print(job.get_attribute("data-job-name"))
    
    time.sleep(2)
    nextPageBtn = driver.find_element(By.CLASS_NAME, "js-next-page")
    nextPageBtn.click()
    
    
# Save the results
dfJobs = pd.DataFrame(jobs)
dfJobs.to_csv("test.csv")

# end
driver.close()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=121.0.6167.184)
Stacktrace:
0   chromedriver                        0x00000001047ee814 chromedriver + 4040724
1   chromedriver                        0x00000001047e6a18 chromedriver + 4008472
2   chromedriver                        0x00000001044596a8 chromedriver + 284328
3   chromedriver                        0x000000010449e330 chromedriver + 566064
4   chromedriver                        0x00000001044934b8 chromedriver + 521400
5   chromedriver                        0x0000000104492ee8 chromedriver + 519912
6   chromedriver                        0x00000001044d7048 chromedriver + 798792
7   chromedriver                        0x000000010449174c chromedriver + 513868
8   chromedriver                        0x0000000104492044 chromedriver + 516164
9   chromedriver                        0x00000001047b3a3c chromedriver + 3799612
10  chromedriver                        0x00000001047b7f1c chromedri

In [27]:
testStr = "AI Software Intern 人工智慧軟體實習生 (台北)"
jobDescription in testStr.split(" ")

False

In [38]:
dfJobs = pd.DataFrame(jobs)
dfJobs
# dfJobs.to_csv("./test.csv")

,name,company
0,AI Software Intern 人工智慧軟體實習生 (台北),立達軟體科技股份有限公司
1,"Software Engineer, Intern / Part-time",臺灣發展軟體科技股份有限公司
2,Software R&amp;D intern (應屆畢業生有機會轉正聘用),美商海鸚科技股份有限公司台灣分公司
3,Software Engineer Intern / 軟體開發實習生,自由食間_滿食空股份有限公司
4,Software Engineer Dev Intern (12 months at lea...,英屬開曼群島商庫幣科技有限公司台灣分公司
...,...,...
196,Software Product Manager-Broadband (Tucheng/Hs...,鴻海精密工業股份有限公司
197,Software Quality Assurance Engineer (R-Sanxia),音律電子股份有限公司
198,AI Engineer (Computer Vision and Generative AI...,訊連科技股份有限公司
199,Software Engineer,晶心科技股份有限公司
